# **A Simple Linear Regression Model Notebook for the Wallaroo Platform**
#### A Comprehensive Tutorial for:
1. Building a Linear Regression Model
2. Deploying the Model into Wallaroo
3. Using Wallaroo's Monitoring Capabilities to Analyze the Model.

### The data was used from GC Wkshps

https://archive.ics.uci.edu/ml/datasets/Room+Occupancy+Estimation

Adarsh Pal Singh, Vivek Jain, Sachin Chaudhari, Frank Alexander Kraemer, Stefan Werner and Vishal Garg, "Machine Learning-Based Occupancy Estimation Using Multivariate Sensor Nodes," in 2018 IEEE Globecom Workshops (GC Wkshps), 2018. 

## **Building a Simple Linear Regression Model**

This model will be built from a data set of sensor values and the occupancy of the room the sensors are in.
The goal is to use the sensor data to predict the room occupancy.

### **Importing the Necessary Python Libraries**

We will use a variety of libraries to implement the linear regression model

#### These libraries include:
- matplotlib
- numpy
- sklearn
- pandas
- onnx

In [ ]:
# Code Source: Us

# Needed for data visualization
import matplotlib.pyplot as plt

# Needed for data tuning
import numpy as np

# Needed for creating the linear regression model
from sklearn import linear_model

# Needed for metrics of the model
from sklearn.metrics import mean_squared_error, r2_score

# Needed for csv importing
import pandas as pd

### **Importing the Data Set using Pandas**
The first step creating a linear regression model is read in the dataset using the pandas library  
The `read_csv` method is responsible for reading in the data and `head()` method acesses the first few rows in the data  
When picking a variable from the data we'll use `.corr()` to find which variable has the best correlation in the dataset

In [ ]:
# Reading and displaying the dataset
data = pd.read_csv('Occupancy_Estimation.csv')
data.head()

In [ ]:
# Displays the correlations between each and every variables
correlations = data.corr()
correlations[-1:]

### **Pulling our Independent (x) and Dependent (y) Variables from the DataFrame**
Next we are going to access the Independent variable `S1_Light` and Dependent variable `Room_Occupancy_Count` to be stored in the x and y value  
The `values` function accesses the values in the dataset at the index of the given variable name

In [ ]:
# Matrix of features x and prints data x
x = data[['S1_Light']].values
print('S1_Light Values:')
print(x)

# Array of independent variable y and prints data y
y = data['Room_Occupancy_Count'].values
print('\nRoom_Occupancy_Count Values:')
print(y)

### **Splitting the Data Set into Testing and Training Subsets**
Next we use the `train_test_split()` method in the sklearn library to split the data into test and train sets 
The train_test_split gives test/train data to x and y  
Create test and train datasets with 0.2 (20%) of the dataset being test data  
The `random_state` decides which indices of data to pull from  


In [ ]:
from sklearn.model_selection import train_test_split

# The train_test_split gives test/train data to x and y
# Create test and train datasets with 0.2 (20%) of the dataset being test data
# The random_state decides which indices of data to pull from
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 3)

### **Creating the Linear Regression Model using Sklearn and Fitting our Training Data to the Model**
Now using `LinearRegression()` method we create a linear regression object which we call `regr`  
Then we take `.fit(x_train, y_train)` method uses x and y train data as parameters to see how well it fits the model

In [ ]:
from sklearn.linear_model import LinearRegression

# Creating the linear regression object
regr = LinearRegression()

# The regr.fit() measures how well the x and y train data fit the model
regr.fit(x_train, y_train)

### **Predicting the Room Occupancy from our Independent Variable Test Set**
In this step we take in the independent variable test set for a the parameter in the `predict()` method in order to predict the outcome for the dependent variable.

In [ ]:
# The regr.predict() creates a prediction based on the x test data
y_pred = regr.predict(x_test)

### **Finding the Metrics to Analyze the Prediction**
In this step we use various functions and methods in order to see how well our linear regression model is predicting our data. The  `coef_` function tells us the **correlation coefficient**, which shows in what way our variables correlate with each other. Next up we have the `mean_squared_error()` method, which shows us the distance from the estimated values and the true values; The best possible score would be 0. Lastly there's the `r2_score()` method which is responsible for displaying how well our data fits the current model, with an R^2 score of 1.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Prints the coefficients
print("Coefficients: \n", regr.coef_)

# Prints the mean squared error
print("Root mean squared error: %.2f" % mean_squared_error(y_test, y_pred, squared=False))

# Prints the coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

### **Plotting the Linear Regression**
Using `plt` in **matplotlib** library we can print out the different aspects of our linear regression graph.

In [ ]:
# The plt.scatter() plots the x and y test points in the linear regression model
plt.scatter(x_test, y_test, color="black")

# The plt.plot() creates the line of best fit
plt.plot(x_test, y_pred, color="blue", linewidth=3)

# Sets the x-axis, y-axis, and title of the model
plt.xlabel('S1_Light')
plt.ylabel('Room Occupancy')
plt.title('Room Occupancy VS S1_Light Sensor')

# The plt.show() displays the model
plt.show()

## **Deploying the Model into Wallaroo**

### **Converting the Sklearn Model into Onnx for use on the Wallaroo Platform**
For the next step refer to [sklearn-regression-to-onnx tutorial](https://docs.wallaroo.ai/wallaroo-tutorials/conversion-tutorials/sklearn-regression-to-onnx/) in the wallaroo documentation for how to convert file to onnx.

In [ ]:
# Used for the conversion process
import onnx, skl2onnx, onnxmltools
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx.common.data_types import DoubleTensorType

In [ ]:
# The model_to_onnx converts the model to onnx to be upload to Wallaroo Platfrom
# For more detailed steps refer to "model_conversion"
def model_to_onnx(model, cols, *, input_type='Double'):
    input_type_lower=input_type.lower()
    # How to manage float values
    if input_type=='Double':
        tensor_type=DoubleTensorType
    elif input_type=='Float':
        tensor_type=FloatTensorType
    else:
        raise ValueError("bad input type")
    tensor_size=cols
    initial_type=[(f'{input_type_lower}_input', tensor_type([None, tensor_size]))]
    onnx_model=onnxmltools.convert_sklearn(model,initial_types=initial_type)
    return onnx_model

In [ ]:
# The model_to_onnx() takes the pickle file and converts it to onnx
onnx_model_converted = model_to_onnx(regr, 1)

# The onnx.save_model() saves the converted model into a file
onnx.save_model(onnx_model_converted, "occupancy_model.onnx")

### Implementing into Wallaroo
Reference the [Wallaroo 101 Tutorial](https://docs.wallaroo.ai/wallaroo-101/) for how to access the wallaroo platform

In [ ]:
# Needed for the use of Wallaroo
import wallaroo

# The wallaroo.Client() allows the file to access wallaroo platform
wl = wallaroo.Client()

In [ ]:
# Creates the name for workspace, pipeline, and model
workspace_name = 'msuproject3'
pipeline_name = 'occupancypipeline2'
model_name = 'occupancymodel2'

# Created to fetch the model
model_file_name = 'occupancy_model.onnx'

In [ ]:
# The get_workspace() gets/create the workspace when needed
# For more detailed steps refer to "wallaroo-101"
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

# The get_pipeline() gets/create the pipeline when needed
# For more detailed steps refer to "wallaroo-101"
def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [ ]:
# Calls function to create workspace
workspace = get_workspace(workspace_name)

# The wl.set_current_workspace() sets the workspace to currently being worked on
set_workspace = wl.set_current_workspace(workspace)

In [ ]:
# Thw wl.list_workspaces() prints the lists of the workspaces
wl.list_workspaces()

In [ ]:
wl.set_current_workspace(workspace)
#gw = wl.get_current_workspace()
# wl.list_models()

### Creating a Pipeline and Uploading the Model
In this step we are using `build_pipeline()`. Here we create the pipeline by giving the method a string. We defined `pipeline_name` earlier.

To upload the model we use `upload_model()`. Here we need give a string, and a file. Both we defined earlier in the tutorial.

Lastly, we add the model as a step to the pipeline using `add_model_step()`. All we have to give here is give the function our model we used earlier.

In [ ]:
# The wl.get_pipeline() creates the pipeline
occupancy_pipeline = wl.build_pipeline(pipeline_name)

# The wl.upload_model() uploads the model to the platform
occupancy_model = wl.upload_model(model_name, model_file_name).configure()

# The occuupancy_pipeline.add_model_step() adds the model to pipeline to be deployed
occupancy_pipeline = occupancy_pipeline.add_model_step(occupancy_model)

### Adding Validation to the model
The `add_validation()` takes in two parameters.  

**add_validation**(**string** name, **bool** condition)  

**Parameters:**
* **Name**: This must be lower-case. It is an arbitrary name used to name the condition.  
* **Condition**: This describes the condition you want to **NOT** mark as an anomaly. First, give the output values and if greater/less/equal to a chosen value.  

The validation step must be implemented before deploying the pipeline and before adding a post process step (In the current version of Wallaroo, this is a work-around to a known issue).  

In our code, we name our validation as `no_negative_people`. The condition takes in our `occupancy_model` outputs and compares it to the `float(0)`. This needs to be a float because `occupancy_model` will output floats. So we are only allowing the values greater than or equal to 0.0 to not be marked as an anomaly.


In [ ]:
occupancy_pipeline = occupancy_pipeline.add_validation(
                                        'no_negative_people',
                                        occupancy_model.outputs[0][0] >= float(0)
                                        )

### Adding Post-Process
The post process step formats the data. We are able to implemennt whatever formatting rules we want.

In this tutorial, since we cannot have a fraction of a person, we implement rounding.

Here we are uploading another "model" to our pipeline.

First, we must take our `postproccess.py` file and upload it to Wallaroo using `wl.upload_model()`. We are naming this model `postprocess`. We are also sending our `postprocess.py` file. This time we must also configure it as a python file. 

We then need to add the model to our pipeline using `add_model_step()` as we did before with our model.
Except this time, we upload `module_post`.

In [ ]:
module_post = wl.upload_model("postprocess", "./postprocess.py").configure('python')

occupancy_pipeline = occupancy_pipeline.add_model_step(module_post)

### Deploying the Model
We are close to the final step now. Here we utilize Wallaroo's amazing cluster to run our pipeline.

In [ ]:
# The occupancy_pipeline.deploy() activating the pipeline
occupancy_pipeline.deploy()

### Checking the deployment
This can be a usefull line of code, to ensure that the pipeline is running succesfully!

In [ ]:
# The occupancy_pipeline.status() displays the status of the pipeline
occupancy_pipeline.status()

### Making an Inference

Finally, we get to utilize all of our work and put it into production.

In this first box, we are just implementing a function that will format our data into a dictionary format. This format is needed in order to properly run inferences.

In [ ]:
# Needed for the infrences
import json
from wallaroo.object import EntityNotFoundError

# The pandas_to_dict() converts the values into dictionary for infrences
def pandas_to_dict(df):
    input_dict = {
    'tensor': df.to_numpy().tolist()
    }
    return input_dict

### Extracting data

Here we just need to grab the data from our .csv that we want to infer upon.

### Getting Single Datums

In our setting, we will pretend that our sensors are sending our pipeline a single datum at a time. In our scenrio, we will send a good datum (something that won't trigger a validation error) and we will also send a bad datum (something that will trigger a validation error).

In [ ]:
good_data = pandas_to_dict(pd.read_csv('Good_Datum.csv').iloc[:,6:7])
bad_data = pandas_to_dict(pd.read_csv('Bad_Datum.csv').iloc[:,6:7])
#sneak_data = pandas_to_dict(pd.read_csv('All_Data_with_sneaky_sneak.csv').iloc[:,6:7])

print(good_data)
print(sneak_data)

### Making the inference

`occupancy_pipeline.infer()`  
This method takes our input data that we pulled from our data, and creates the inference. 

The result shows the data we placed in, the outputs from the inference, and lastly, our post process step that rounds the numbers.

`result[0].data()[0].tolist()`  
This makes the result a little easier to see.

In [ ]:
# The occupancy_pipeline.infer() creates a result based on data given
good_result = occupancy_pipeline.infer(good_data)
good_result

print("Finalized Data")
good_result[0].data()[0].tolist()

In [ ]:
# The occupancy_pipeline.infer() creates a result based on data given
bad_result = occupancy_pipeline.infer(bad_data)
bad_result

print("Finalized Data")
bad_result[0].data()[0].tolist()

In [ ]:
# The occupancy_pipeline.infer() creates a result based on data given
# sneak_bad_result = occupancy_pipeline.infer(sneak_data)
# sneak_bad_result

# print("Finalized Data")
# sneak_bad_result[0].data()[0].tolist()

### Undeploy the pipeline

This part is crucial, you do not want take up more resources than you meant to. 

In [ ]:
# The occupancy_pipeline.undeploy() deactivates the pipeline
occupancy_pipeline.undeploy()

### Logs

`.logs()`

Lastly, the pipeline logs can be deployed, even after the pipeline is undeployed.

In [ ]:
logs = occupancy_pipeline.logs()
logs